In [134]:
import pandas as pd

import json
from pprint import pprint

#Use gensim, which remove stop words, short words(<3 letters), puncts and do stemming
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string

# train_text=preprocess_documents(text)

from collections import Counter
import numpy as np

# mapped_text

from gensim.models import AuthorTopicModel
from gensim.corpora import mmcorpus
from gensim.test.utils import common_dictionary, datapath, temporary_file
# author2doc = {
#     'john': [0, 1, 2, 3, 4, 5, 6],
#     'jane': [2, 3, 4, 5, 6, 7, 8],
#     'jack': [0, 2, 4, 6, 8]
# }
# corpus = mmcorpus.MmCorpus(datapath('testcorpus.mm'))
# with temporary_file("serialized") as s_path:
#     model = AuthorTopicModel(
#          corpus, author2doc=author2doc, id2word=common_dictionary, num_topics=4,
#          serialized=True, serialization_path=s_path
#     )
#     model.update(corpus, author2doc)  # update the author-topic model with additional documents
# author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

from gensim import corpora

## json to df

In [135]:
import pickle as pkl
with open('states.pkl','rb') as file:
    states=pkl.load(file)

In [136]:
import json 
with open('firstdebat_cleaned.json','r') as json_file:
    text=json.load(json_file)

In [137]:
t_ids=list(text['full_text'].keys())

In [138]:

text['full_text']['1033']

'@realDonaldTrump @KellyannePolls Russian ?s:no relation w/Putin: but (HRC) allowed a Russian co. buy 20% US uranium prod 4 WJC speech pay.'

In [139]:
states_full=set(list(states.values()))

In [140]:
states_abb=set(list(states.keys()))

In [141]:
ID2loc={}
for ID in t_ids:
    loc=text['place'][ID][4].split()
    if len(loc)>1:
        if loc[0] in states_full:
            ID2loc[ID]=loc[0]
        elif loc[1] in states_abb:
            ID2loc[ID]=states[loc[1]]
        
#     print(text['place'][ID][4])

In [142]:
valid_ids=set(ID2loc.keys())

In [143]:
ID2text={}
for ID in t_ids:
    sent=text['full_text'][ID]
    if ID in valid_ids:
        ID2text[ID]=sent

In [144]:
IDs=[]
sents=[]
locs=[]
for ID in valid_ids:
    IDs.append(ID)
    sents.append(ID2text[ID])
    locs.append(ID2loc[ID])

In [145]:
df={'ID':IDs,'text':sents,'state':locs}

In [146]:
odf=pd.DataFrame.from_dict(df)

## df to gensim corpus

In [147]:
documents=odf['text'].values

In [152]:
documents[5]

"@HillaryClinton @ESS421 No thanks.  #NoMoreBigGovt #LetGaryDebate  I'm voting 3rd party. @GovGaryJohnson"

In [120]:
# filtered_docs=preprocess_documents(documents, CUSTOM_FILTERS)

In [155]:
filtered_docs=list(map(lambda t:preprocess_string(t, CUSTOM_FILTERS),documents))

In [154]:
from gensim.parsing.preprocessing import *
CUSTOM_FILTERS = [lambda x: x.lower(),strip_multiple_whitespaces,remove_stopwords, strip_non_alphanum,strip_numeric,lambda x: strip_short(x,minsize=3)]
preprocess_string(documents[5], CUSTOM_FILTERS)

['hillaryclinton',
 'ess',
 'thanks',
 'nomorebiggovt',
 'letgarydebate',
 'voting',
 'party',
 'govgaryjohnson']

In [156]:
# odf['text']=filtered_docs
texts=filtered_docs

In [157]:
# remove common words and tokenize
# stoplist = set('for a of the and to in'.split())
# texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 0] for text in texts]

from pprint import pprint  # pretty-printer
# pprint(texts)

In [78]:
# corpus

In [158]:
import os
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join('temp.dict'))

corpus = [dictionary.doc2bow(text) for text in texts]

In [159]:
n=0
author2doc={}
for author in states.values():
    author2doc[author]=[]

for author in odf['state'].values:
    author2doc[author].append(int(n))
    n+=1

In [160]:
dictionary

In [177]:
# author2doc = {
#     'john': [0, 1, 2, 3, 4, 5, 6],
#     'jane': [2, 3, 4, 5, 6, 7, 8],
#     'jack': [0, 2, 4, 6, 8]
# }
# corpus = mmcorpus.MmCorpus(datapath('testcorpus.mm'))
# with temporary_file("serialized") as s_path:

n_topics=5
model = AuthorTopicModel(
     corpus, author2doc=author2doc, id2word=dictionary, num_topics=n_topics,
     serialized=False,eval_every=10, iterations=100
)
model.update(corpus, author2doc)  # update the author-topic model with additional documents
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

In [76]:
# model.author2id

In [166]:
[dictionary[pair[0]] for pair in model.get_topic_terms(5, topn=30)]

['https',
 'debates',
 'vote',
 'think',
 'now',
 'winning',
 'told',
 'cast',
 'nbcwashington',
 'nbcdc',
 'debatenight',
 'realdonaldtrump',
 'hillaryclinton',
 'trump',
 'debate',
 'hillary',
 'tonight',
 'you',
 'like',
 'amp',
 'donald',
 'let',
 'imwithher',
 'people',
 'can',
 'great',
 'lol',
 'won',
 'clinton',
 'know']

In [73]:
len(author_vecs)

61

In [168]:
dists=[]

for dist in author_vecs:
    new_dist=np.zeros(n_topics)
    for pair in dist:
        new_dist[pair[0]]=pair[1]
    dists.append(new_dist)

In [170]:
topic_dist=pd.DataFrame(dists)

In [171]:
topic_dist['state']=list(model.id2author.values())

In [173]:
topic_dist.sort_values(by=[5],ascending=False)

,0,1,2,3,4,5,state
40,0.023986,0.060377,0.021238,0.000000,0.037009,0.853371,Oklahoma
54,0.036953,0.015012,0.000000,0.000000,0.135052,0.798718,West Virginia
12,0.050542,0.041401,0.038152,0.064339,0.043228,0.762337,Guam
30,0.000000,0.038695,0.032444,0.071352,0.138502,0.711995,Nebraska
26,0.000000,0.088886,0.077524,0.000000,0.138419,0.690080,Mississippi
0,0.023376,0.068503,0.114633,0.057710,0.062325,0.673454,Alabama
16,0.000000,0.035745,0.109049,0.000000,0.222836,0.625301,Indiana
20,0.148801,0.032700,0.067749,0.000000,0.142230,0.606696,Louisiana
39,0.015578,0.253924,0.024766,0.017436,0.123152,0.565144,Ohio
47,0.031109,0.220792,0.045321,0.000000,0.178389,0.522985,Tennessee


In [59]:
# from scipy.sparse import csr_matrix
# csr_matrix

In [72]:
list(author2doc.keys())

['Alaska',
 'Alabama',
 'Arkansas',
 'American Samoa',
 'Arizona',
 'California',
 'Colorado',
 'Connecticut',
 'District of Columbia',
 'Delaware',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Iowa',
 'Idaho',
 'Illinois',
 'Indiana',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Massachusetts',
 'Maryland',
 'Maine',
 'Michigan',
 'Minnesota',
 'Missouri',
 'Northern Mariana Islands',
 'Mississippi',
 'Montana',
 'National',
 'North Carolina',
 'North Dakota',
 'Nebraska',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'Nevada',
 'New York',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Virginia',
 'Virgin Islands',
 'Vermont',
 'Washington',
 'Wisconsin',
 'West Virginia',
 'Wyoming']